# `swift_too` module

## Swift_VisQuery Example - Querying the visibility for a given target

### API Version 1.2

#### Author: Jamie A. Kennea (Penn State)

### Introduction 

This example shows how we can use the `swift_too` module to give us predicted long term visibility for targets. Note that long term visibility calculates if a target can be observed on Swift based on Sun, Moon and Pole constraints. It does not calculate the visibility due to Earth occultations. 

Visibility is based on the most up to date TLE derived prediction of the spacecraft location into the future. Although this will not affect Sun and Moon constraints, Pole constraint accuracy may get worse the further into the future we look.

### Setting up the Query

First step, import the `Swift_VisQuery` class from TOO API, and whatever supporting Python modules we might need along the way.

In [1]:
from swifttools.swift_too import Swift_VisQuery
from datetime import datetime,timedelta
from time import sleep
import numpy as np

Next we'll set up our username and shared_secret. No need to do this in variables, but we'll use them more than once, so it's easier this way.

In [2]:
username = "myuser"
shared_secret = "mysharedsecret"

Next, we set up the class and set our username and shared_secret. All TOO API submissions require a validated username and shared_secret, so we can track who made the submissions and verify that they were from that person.

In [3]:
query = Swift_VisQuery()
query.username = username
query.shared_secret = shared_secret

### Coordinate formats

We set up the parameters for the query, the minimum required parameters are RA/Dec coordinates, and the number of days to calculate the visibility over in days. RA/Dec are given in decimal degrees in J2000.

In [4]:
query.ra, query.dec = 113.771875, -69.131472

Or for convenience we can use Astropy SkyCoord object to submit the coordinates in any format you like. They will just be converted into J2000 / FK5 RA/Dec upon submission. The following example shows how to submit a request based on Galactic Coordinates, in our case, the Galactic Center (lII/bII = 0,0).

In [5]:
from astropy.coordinates import SkyCoord
query.skycoord = SkyCoord('0.0', '0.0', frame='galactic', unit='deg')

If we print out RA / Dec now, we'll see it reflects the SkyCoord in the previous line.

In [6]:
print(f"SkyCoord should be the equivalent of RA/Dec(J2000) = {query.ra:.4f}, {query.dec:.4f}")

SkyCoord should be the equivalent of RA/Dec(J2000) = 266.4050, -28.9362


Note if you modify the RA/Dec values after setting a skycoord, the skycoord variable is altered to match the new coordinates. Note that the frame will default to FK5. This is a neat trick, but it must be noted that unless you pass a SkyCoord, TOO_API will not return one. This is because the Swift TOO API does not depend on astropy. However if you do use SkyCoords, then compatibility is assured!

In [7]:
query.ra = 113.7718769
query.dec = -69.1314721 + np.random.random()*1
print(query.skycoord)

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (113.7718769, -68.71338971)>


### Visibility Period

Final parameter, we set the length of time we wish to calculate the visibility for. Note we haven't set a start time, it will default to now.

In [8]:
query.length = 7 # days

### Submitting the Query for processing

Now we submit the request to the Swift TOO API server.

In [9]:
if query.queue():
    while(not query.complete):
        print(f"Waiting for job #{query.status.jobnumber} to be processed ...")
        sleep(2)
    print(f"Done. Job #{query.status.jobnumber} complete")
else:
    print(f"Request rejected. Error: {query.status.errors}")

Done. Job #47600 complete


In [10]:
if query.submit():
    print("Success!")
else:
    print("Failure")

Success!


### Examining the returned results

Assuming success let's see what we get from the TOO API server in response to this request. We'll do this by printing out api_data which is a dictionary containing the both the input parameters and the returned parameters.

In [11]:
query.api_data

{'username': 'myuser',
 'ra': 113.7718769,
 'dec': -68.71338971439681,
 'begin': '2021-07-06 16:59:04.310320',
 'length': 7,
 'hires': False,
 'windows': [{'api_name': 'Swift_VisWindow',
   'api_version': '1.2',
   'api_data': {'begin': '2021-07-06 16:59:00',
    'end': '2021-07-13 16:59:00'}}]}

We can see here that the results are encoded within the parameter "windows", which contains the entries encoded in the Swift_VisWindow class. This simple class just contains the start and end of the window. We can access this by simply indexing the query.


In [12]:
_ = [print(window) for window in query]

2021-07-06 16:59:00 - 2021-07-13 16:59:00 (7 days, 0:00:00)


Or we can get this by simply typing in the name of the class in this Jupyter Notebook.

In [13]:
query

Begin,End,Length
2021-07-06 16:59:00,2021-07-13 16:59:00,"7 days, 0:00:00"


You'll notice the API starts calculating the visibility from when the submission was sent, or at least to the nearest minute, if the `begin` property is not set. If the target is still visibile window after the amount of time you specified by `length` has ended, it will report that as the end of the visibility window, because it does not calculate beyond that.

In [14]:
print(f"Time between start of first window and end of last window = {query[-1][1] - query[0][0]}")

Time between start of first window and end of last window = 7 days, 0:00:00


### Time formats 

Window start / stop times are datetime structures and values are UTC.

In [15]:
query[0].begin

datetime.datetime(2021, 7, 6, 16, 59)

astropy fans can convert these to a Time class easily.

In [16]:
from astropy.time import Time

tbegins = Time([q.begin for q in query], scale='utc')
tends = Time([q.end for q in query], scale='utc')
tbegins[0]

<Time object: scale='utc' format='datetime' value=2021-07-06 16:59:00>

Which of course makes things like converting into other formats easy!

In [17]:
print(f"MJD Visibility Periods for RA/Dec(J2000) = {query.ra}, {query.dec}:\n")
for i in range(len(tbegins)):
    print(f"MJD {tbegins[i].mjd:.3f} - MJD {tends[i].mjd:.3f}")

MJD Visibility Periods for RA/Dec(J2000) = 113.7718769, -68.71338971439681:

MJD 59401.708 - MJD 59408.708


### High resolution visibility

*Swift* is in a low Earth orbit, so visibility of a target depends on not just the relative positions of the Sun and Moon, but also on whether the target is occulted by the Earth. As Swift's orbit is approximately 96 minutes long, these Earth occultations regularly occur. As you can see from the visibility period above, it does not take these into account. 

In addition *Swift* cannot observe while the spacecraft is passing through the [South Atlantic Anomaly](https://en.wikipedia.org/wiki/South_Atlantic_Anomaly), and so these periods must be blocked out.

You can calculate visiblity windows including these constraints using the `hires` flag. 

Let's set up a new VisQuery, using the hires flag. This time we'll use a more compact argument based to call it. Note that here we explicity set a `begin` and `end` time for the visibility period.

In [18]:
hvq = Swift_VisQuery(username=username,
                     shared_secret=shared_secret,
                     ra=113.771875, dec=-69.131472, 
                     begin=datetime(2021,2,1),
                     end=datetime(2021,2,2), 
                     hires=True)

Note that using this form, we don't have to use the `submit()` method, it is called automatically. Still we should still check that the request was successful.

In [19]:
if hvq.status.status == 'Accepted':
    print("All Good!")
else:
    print(f"Not good: {hvq.status}")

All Good!


OK, let's see how our visibility windows look now:

In [20]:
hvq

Begin,End,Length
2021-02-01 00:33:00,2021-02-01 00:50:00,0:17:00
2021-02-01 00:59:00,2021-02-01 01:12:00,0:13:00
2021-02-01 02:09:00,2021-02-01 02:32:00,0:23:00
2021-02-01 02:41:00,2021-02-01 02:47:00,0:06:00
2021-02-01 03:44:00,2021-02-01 04:14:00,0:30:00
2021-02-01 04:22:00,2021-02-01 04:23:00,0:01:00
2021-02-01 05:20:00,2021-02-01 05:56:00,0:36:00
2021-02-01 06:55:00,2021-02-01 07:34:00,0:39:00
2021-02-01 08:31:00,2021-02-01 09:09:00,0:38:00
2021-02-01 10:06:00,2021-02-01 10:45:00,0:39:00


As you can see, despite only being for a period covering one day, the number of windows is much greater, and the windows are shorter. These windows are the true visibility of a target to *Swift*. Note however, that although windows can be up to 45 minutes long, the maximum that *Swift* can observed a pre-planned target for is 30 minutes.

**Another thing to note**: Maximum length that can be calculated in high resolution is 20 days. This is because it is computationally expensive to make this calculation. In addition, these windows are calculated based on a contemporary *Swift* ephemeris derived from a [Two Line Element](https://en.wikipedia.org/wiki/Two-line_element_set). This means that if you go too far into the future, the accuracy of the orbit prediction will suffer. We therefore recommend that you treat predictions of high resolution visibility that go more than 20 days into the future with caution.